In [1]:
from datetime import datetime, timedelta
import numpy as np
import random
from infrastructure import *
from signaling import *
from system import *
from train import *
from dispatch import *
import networkx as nx
from sim import *
from rail_networkx import *
from itertools import combinations, permutations

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from rail_networkx import all_simple_paths, shortest_path

In [2]:
sim_init_time = datetime.strptime('2018-01-10 10:00:00', "%Y-%m-%d %H:%M:%S")
sim_term_time = datetime.strptime('2018-01-10 15:30:00', "%Y-%m-%d %H:%M:%S")
spd_container = [random.uniform(0.01, 0.02) for i in range(20)]
acc_container = [0.5*random.uniform(2.78e-05*0.85, 2.78e-05*1.15) for i in range(20)]
dcc_container = [0.2*random.uniform(2.78e-05*0.85, 2.78e-05*1.15) for i in range(20)]
headway = 300 + random.random() * 400
sys = System(sim_init_time, spd_container, acc_container, dcc_container,
             term_time=sim_term_time,
             dos_period=['2018-01-10 11:30:00', '2018-01-10 12:30:00'],  
             dos_pos=(15,20),
             headway=headway, 
             refresh_time=50)
dp = Dispatcher(sys)
K165 = Train(sys, 
             ((sys.signal_points[0],1),(sys.signal_points[1],0)),
             ((sys.signal_points[10],1),(None,None)),
             max_spd=random.choice(sys.spd_container),
             max_acc=random.choice(sys.acc_container),
             max_dcc=random.choice(sys.dcc_container),
             length=1)
T165 = Train(sys, 
             ((None,None),(sys.signal_points[0],0)),
             ((sys.signal_points[10],1),(None,None)),
             max_spd=K165.max_spd*1.02,
             max_acc=random.choice(sys.acc_container),
             max_dcc=random.choice(sys.dcc_container),
             length=1)
K166 = Train(sys, 
             ((sys.signal_points[10],0),(sys.signal_points[9],1)),
             ((sys.signal_points[0],0),(None,None)),
             max_spd=random.choice(sys.spd_container),
             max_acc=random.choice(sys.acc_container),
             max_dcc=random.choice(sys.dcc_container),
             length=1)
T166 = Train(sys, 
             ((None,None),(sys.signal_points[10],1)),
             ((sys.signal_points[0],0),(None,None)),
             max_spd=K166.max_spd*1.02,
             max_acc=random.choice(sys.acc_container),
             max_dcc=random.choice(sys.dcc_container),
             length=1)

In [7]:
while sys.sys_time - sys.init_time <= sys.term_time - sys.init_time:
    _semaphore_to_return = False
    for t in sys.trains:
        t.request_routing()
        t.update_acc()
    sys.sys_time+=sys.refresh_time
    if K165.curr_MP > 20:
        break

CtrlPnt 2 CtrlPnt 3
CtrlPnt 2 CtrlPnt 4
CtrlPnt 2 CtrlPnt 5
CtrlPnt 2 CtrlPnt 6
CtrlPnt 2 CtrlPnt 7
CtrlPnt 2 CtrlPnt 8
CtrlPnt 2 CtrlPnt10
CtrlPnt 2 CtrlPnt13
CtrlPnt 2 CtrlPnt16
train idx: 0 occupying:[((CtrlPnt 0, 1), (AutoPnt 1, 0))] head MP: 5.00 rear MP: 4.00, requested (0, 1) at 10.0
	route (0, 1) of CtrlPnt 2 is opened
CtrlPnt 0 CtrlPnt 2
CtrlPnt 0 CtrlPnt 3
CtrlPnt 0 CtrlPnt 4
CtrlPnt 0 CtrlPnt 5
CtrlPnt 0 CtrlPnt 6
CtrlPnt 0 CtrlPnt 7
CtrlPnt 0 CtrlPnt 8
CtrlPnt 0 CtrlPnt10
CtrlPnt 0 CtrlPnt13
CtrlPnt 0 CtrlPnt14
CtrlPnt 0 CtrlPnt16
train idx: 1 occupying:[((None, None), (CtrlPnt 0, 0))] head MP: 0.00 rear MP:-1.00, requested (0, 1) at 0.0
	route (0, 1) of CtrlPnt 0 is opened
CtrlPnt 8 CtrlPnt 0
CtrlPnt 8 CtrlPnt 2
CtrlPnt 8 CtrlPnt 3
CtrlPnt 8 CtrlPnt 4
CtrlPnt 8 CtrlPnt 5
CtrlPnt 8 CtrlPnt 6
CtrlPnt 8 CtrlPnt 7
CtrlPnt 8 CtrlPnt13


AttributeError: 'NoneType' object has no attribute 'pop'

In [3]:
print(sys.signal_points[0].signal_by_port[0].route)
print(sys.signal_points[0].signal_by_port[0].number_of_blocks_cleared_ahead)
print(sys.signal_points[0].signal_by_port[0].aspect)
print(sys.signal_points[0].all_valid_routes)
print(sys.signal_points[0].available_ports_by_port)
print(sys.signal_points[0].ban_ports_by_port)

None
0
Aspect: r, 	 route None, target speed 0.0 mph
[(0, 1), (1, 0)]
defaultdict(<class 'list'>, {0: [1], 1: [0]})
{0: [0], 1: [1]}


In [12]:
path = list(all_simple_paths(sys.G_skeleton, sys.signal_points[4], sys.signal_points[6]))
print(path[0])

NameError: name 'all_simple_paths' is not defined

In [4]:
dp.get_route(sys.signal_points[2], 0, sys.signal_points[16], 1)

[((CtrlPnt 0, 1), (CtrlPnt 2, 0)),
 ((CtrlPnt 2, 3), (CtrlPnt14, 0)),
 ((CtrlPnt14, 1), (CtrlPnt16, 0)),
 ((CtrlPnt16, 1), (CtrlPnt13, 0))]

In [5]:
[b for b in sys.bigblocks]

[BgBlk <MP:  0.0~10.0 > <CtrlPnt 0 port: 1~CtrlPnt 2 port: 0> key: 0,
 BgBlk <MP: 10.0~15.0 > <CtrlPnt 2 port: 1~CtrlPnt 3 port: 0> key: 0,
 BgBlk <MP: 10.0~5.0  > <CtrlPnt 2 port: 3~CtrlPnt14 port: 0> key: 0,
 BgBlk <MP: 15.0~20.0 > <CtrlPnt 3 port: 1~CtrlPnt 4 port: 0> key: 0,
 BgBlk <MP: 15.0~20.0 > <CtrlPnt 3 port: 3~CtrlPnt 4 port: 2> key: 1,
 BgBlk <MP: 20.0~25.0 > <CtrlPnt 4 port: 1~CtrlPnt 5 port: 0> key: 0,
 BgBlk <MP: 25.0~30.0 > <CtrlPnt 5 port: 1~CtrlPnt 6 port: 0> key: 0,
 BgBlk <MP: 25.0~40.0 > <CtrlPnt 5 port: 3~CtrlPnt 8 port: 2> key: 0,
 BgBlk <MP: 30.0~35.0 > <CtrlPnt 6 port: 1~CtrlPnt 7 port: 0> key: 0,
 BgBlk <MP: 30.0~35.0 > <CtrlPnt 6 port: 3~CtrlPnt 7 port: 2> key: 1,
 BgBlk <MP: 35.0~40.0 > <CtrlPnt 7 port: 1~CtrlPnt 8 port: 0> key: 0,
 BgBlk <MP: 40.0~50.0 > <CtrlPnt 8 port: 1~CtrlPnt10 port: 0> key: 0,
 BgBlk <MP: 15.0~20.0 > <CtrlPnt16 port: 1~CtrlPnt13 port: 0> key: 0,
 BgBlk <MP:  5.0~15.0 > <CtrlPnt14 port: 1~CtrlPnt16 port: 0> key: 0,
 BgBlk <MP:  5.0~15.

In [3]:
for e in sys.G_skeleton.edges(data=True):
    print(e[2]['instance'].tracks)

[Track <MP:  0.0~5.0  > <CtrlPnt 0 port: 1~AutoPnt 1 port: 0> key: 0, Track <MP:  5.0~10.0 > <AutoPnt 1 port: 1~CtrlPnt 2 port: 0> key: 0]
[Track <MP: 10.0~15.0 > <CtrlPnt 2 port: 1~CtrlPnt 3 port: 0> key: 0]
[Track <MP: 10.0~5.0  > <CtrlPnt 2 port: 3~CtrlPnt14 port: 0> key: 0]
[Track <MP: 15.0~20.0 > <CtrlPnt 3 port: 1~CtrlPnt 4 port: 0> key: 0]
[Track <MP: 15.0~20.0 > <CtrlPnt 3 port: 3~CtrlPnt 4 port: 2> key: 1]
[Track <MP: 20.0~25.0 > <CtrlPnt 4 port: 1~CtrlPnt 5 port: 0> key: 0]
[Track <MP: 25.0~30.0 > <CtrlPnt 5 port: 1~CtrlPnt 6 port: 0> key: 0]
[Track <MP: 25.0~30.0 > <CtrlPnt 5 port: 3~AutoPnt11 port: 0> key: 0, Track <MP: 30.0~35.0 > <AutoPnt11 port: 1~AutoPnt12 port: 0> key: 0, Track <MP: 35.0~40.0 > <AutoPnt12 port: 1~CtrlPnt 8 port: 2> key: 0]
[Track <MP: 30.0~35.0 > <CtrlPnt 6 port: 1~CtrlPnt 7 port: 0> key: 0]
[Track <MP: 30.0~35.0 > <CtrlPnt 6 port: 3~CtrlPnt 7 port: 2> key: 1]
[Track <MP: 35.0~40.0 > <CtrlPnt 7 port: 1~CtrlPnt 8 port: 0> key: 0]
[Track <MP: 40.0~45.0 >

In [10]:
for i in sys.ctrl_points:
    for _,j in i.signal_by_port.items():
        print(j)

HomeSig port: 0 of CtrlPnt 0
HomeSig port: 1 of CtrlPnt 0
HomeSig port: 0 of CtrlPnt 2
HomeSig port: 1 of CtrlPnt 2
HomeSig port: 3 of CtrlPnt 2
HomeSig port: 0 of CtrlPnt 3
HomeSig port: 1 of CtrlPnt 3
HomeSig port: 3 of CtrlPnt 3
HomeSig port: 0 of CtrlPnt 4
HomeSig port: 2 of CtrlPnt 4
HomeSig port: 1 of CtrlPnt 4
HomeSig port: 0 of CtrlPnt 5
HomeSig port: 1 of CtrlPnt 5
HomeSig port: 3 of CtrlPnt 5
HomeSig port: 0 of CtrlPnt 6
HomeSig port: 1 of CtrlPnt 6
HomeSig port: 3 of CtrlPnt 6
HomeSig port: 0 of CtrlPnt 7
HomeSig port: 2 of CtrlPnt 7
HomeSig port: 1 of CtrlPnt 7
HomeSig port: 0 of CtrlPnt 8
HomeSig port: 2 of CtrlPnt 8
HomeSig port: 1 of CtrlPnt 8
HomeSig port: 0 of CtrlPnt10
HomeSig port: 1 of CtrlPnt10
HomeSig port: 0 of CtrlPnt13
HomeSig port: 1 of CtrlPnt13
HomeSig port: 0 of CtrlPnt14
HomeSig port: 1 of CtrlPnt14
HomeSig port: 3 of CtrlPnt14
HomeSig port: 0 of CtrlPnt16
HomeSig port: 2 of CtrlPnt16
HomeSig port: 1 of CtrlPnt16


In [11]:
for i in sys.bigblocks:
    print(i)

BgBlk MP:<  0.0~10.0 > <CtrlPnt 0 port: 1~CtrlPnt 2 port: 0> key: 0
BgBlk MP:< 10.0~15.0 > <CtrlPnt 2 port: 1~CtrlPnt 3 port: 0> key: 0
BgBlk MP:< 10.0~5.0  > <CtrlPnt 2 port: 3~CtrlPnt14 port: 0> key: 0
BgBlk MP:< 15.0~20.0 > <CtrlPnt 3 port: 1~CtrlPnt 4 port: 0> key: 0
BgBlk MP:< 15.0~20.0 > <CtrlPnt 3 port: 3~CtrlPnt 4 port: 2> key: 1
BgBlk MP:< 20.0~25.0 > <CtrlPnt 4 port: 1~CtrlPnt 5 port: 0> key: 0
BgBlk MP:< 25.0~30.0 > <CtrlPnt 5 port: 1~CtrlPnt 6 port: 0> key: 0
BgBlk MP:< 25.0~40.0 > <CtrlPnt 5 port: 3~CtrlPnt 8 port: 2> key: 0
BgBlk MP:< 30.0~35.0 > <CtrlPnt 6 port: 1~CtrlPnt 7 port: 0> key: 0
BgBlk MP:< 30.0~35.0 > <CtrlPnt 6 port: 3~CtrlPnt 7 port: 2> key: 1
BgBlk MP:< 35.0~40.0 > <CtrlPnt 7 port: 1~CtrlPnt 8 port: 0> key: 0
BgBlk MP:< 40.0~50.0 > <CtrlPnt 8 port: 1~CtrlPnt10 port: 0> key: 0
BgBlk MP:< 15.0~20.0 > <CtrlPnt16 port: 1~CtrlPnt13 port: 0> key: 0
BgBlk MP:<  5.0~15.0 > <CtrlPnt14 port: 1~CtrlPnt16 port: 0> key: 0
BgBlk MP:<  5.0~15.0 > <CtrlPnt14 port: 3~CtrlPn